# Resources
1) https://www.tensorflow.org/text/guide/word_embeddings
2) https://projector.tensorflow.org/?_gl=1*1j4i7zy*_ga*MTczNjg0ODUwMy4xNzIwODcxNzY4*_ga_W0YLR4190T*MTc0NTk1MTgxMi4yOC4xLjE3NDU5NTI4MDQuMC4wLjA.
3) Jay Alammar

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Intent/intent_Train.csv')
df_valid = pd.read_csv('Intent/intent_Valid.csv')
df_test = pd.read_csv('Intent/intent_Test.csv')

In [3]:
df_test.head()

,utterance,intent,category,tags
0,would it be possible to cancel the order I made?,cancel_order,ORDER,BIP
1,cancelling order,cancel_order,ORDER,BK
2,I need assistance canceling the last order I h...,cancel_order,ORDER,B
3,problem with canceling the order I made,cancel_order,ORDER,B
4,I don't know how to cancel the order I made,cancel_order,ORDER,B


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6539 entries, 0 to 6538
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   utterance  6539 non-null   object
 1   intent     6539 non-null   object
 2   category   6539 non-null   object
 3   tags       6539 non-null   object
dtypes: object(4)
memory usage: 204.5+ KB


In [5]:
df.isnull().sum()

utterance    0
intent       0
category     0
tags         0
dtype: int64

In [6]:
df = df.sample(frac=1, random_state=26)#1 for 100%
df.head()

,utterance,intent,category,tags
3602,I have to correct the details on my user profile,edit_account,ACCOUNT,BLM
3068,I want assistance to check what delivery optio...,delivery_options,DELIVERY,BM
5989,I do not know what I have to do to switch to a...,switch_account,ACCOUNT,BE
3911,would it be possible to get refunds?,get_refund,REFUND,BIMP
5640,I try to set another shipping address up,set_up_shipping_address,SHIPPING_ADDRESS,B


In [7]:
len(df_valid['intent'].unique())

27

# Now we will load all the train test valid dataset and split them

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
df.drop(['category','tags'], axis=1, inplace=True)
df_valid.drop(['category','tags'], axis=1, inplace=True)
# df_valid.head()
df_test.drop(['category','tags'], axis=1, inplace=True)

In [10]:
X_train = df['utterance']
X_valid = df_valid['utterance']
X_test = df_test['utterance']

In [11]:
y_train = df['intent']
y_valid = df_valid['intent']
y_test = df_valid['intent']

In [12]:
X_train

3602     I have to correct the details on my user profile
3068    I want assistance to check what delivery optio...
5989    I do not know what I have to do to switch to a...
3911                 would it be possible to get refunds?
5640             I try to set another shipping address up
                              ...                        
1985                  help me contacting customer service
1456          help me check how long refunds usually take
5894                     assistance using another account
5438    is there a section to submit a review for your...
4917                          help me recover my password
Name: utterance, Length: 6539, dtype: object

# Text Vectoriztion layer 

standarizes each sample
Splits the sentence into words
recombine using ngrams
index the tokens
transform each samples to vector using indexes

In [13]:
import tensorflow as tf

# tf.keras.layers.TextVectorization

In [14]:
from concurrent.futures.process import _ThreadWakeup


# Defualt

text_Vectorizer = tf.keras.layers.TextVectorization (max_tokens= None,  # how many words in vocab
                                                     standardize="lower_and_strip_punctuation",
                                                     split= "whitespace",
                                                     ngrams= None,
                                                     output_mode="int", # can be function but we need int
                                                     output_sequence_length= None # We dont cap the output as longest sequence is unknown it will basically pad others
                                                    #  pad_to_max_tokens=True
)

To keep our data small we need the same size of number of tokens in a list

So now clculating avg length of sentences

In [15]:
# s = 0
# for i in range(len(X_train)):
#     print(len(X_train[i].split()))
#     s = s + len(X_train[i].split()) #lENGTH OF SPLIT
# s = s/len(X_train)
# print("Average length of sequences" , s)

In [16]:
average_length = round(sum(len(i.split()) for i in X_train) / len(X_train))
print("Average length of sequences:", average_length)


Average length of sequences: 8


In [17]:
max_vocab_length = 10000
max_length = average_length #8
#max_length = 8
text_Vectorizer = tf.keras.layers.TextVectorization (max_tokens= max_vocab_length,  # how many words in vocab
                                                    #  standardize="lower_and_strip_punctuation",
                                                    #  split= "whitespace",
                                                    #  ngrams= None,
                                                     output_mode="int", # can be function but we need int
                                                     output_sequence_length= max_length # We dont cap the output as longest sequence is unknown it will basically pad others
                                                    #  pad_to_max_tokens=True
)
 # To fit this instantance we use METHOD ADAPT

# Once adpated we can test
text_Vectorizer.adapt(X_train)

In [18]:
sample = "My name is Vedant Ranade I am Learning"
print(text_Vectorizer(sample))

tf.Tensor([ 8  1 41  1  1  2 73  1], shape=(8,), dtype=int64)


In [19]:
# We can also check details about the vocab
words = text_Vectorizer.get_vocabulary()
print(len(words))
print(f" Most common words {words[:5]}")
print(f" Least common words {words[-5:]}")

658
 Most common words ['', '[UNK]', 'i', 'to', 'the']
 Least common words ['accont', 'accesws', 'accdss', 'abuot', '7']


# Now creating Embiddings using embidding

converts integers to dense vectors of fixed size

we will set input dim: Size of Vocab

output dim : size of output embedding (10 : each token will be represented by a vector 10 long)

Length of Sequence passed

In [20]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,# Input shape or Vocab
                        output_dim=128,
                        input_length=max_length# Length of Sequence
                        )

In [21]:
import random
sample = random.choice(X_train)

In [22]:
print("Original Sentence: ", sample)
sample_embed = embedding(text_Vectorizer([sample]))# First convert to vector then embedding hoga
print("Embedding", sample_embed)
# shape (1(sequence), (padded or truncated)8, (Each token is represented as)128 long vector)

Original Sentence:  i have submitted a wrong delivery address can uupdatte it
Embedding tf.Tensor(
[[[-0.02418038 -0.04768017  0.00464603 ...  0.04623098  0.02109656
    0.02303879]
  [ 0.01298225  0.02786713  0.04385399 ... -0.01711935  0.03946717
    0.03335721]
  [-0.00097357 -0.03916441 -0.04893842 ...  0.00181059  0.03932572
    0.03716135]
  ...
  [ 0.0181983   0.01500782 -0.01071702 ... -0.04168265 -0.00226492
    0.00865587]
  [-0.0272254   0.00243301  0.03431324 ... -0.04759513 -0.00669425
   -0.04339453]
  [ 0.00538235 -0.04420134 -0.0430579  ... -0.03193692  0.02613112
   -0.00785073]]], shape=(1, 8, 128), dtype=float32)


In [23]:
# Encoding the labels that will be used further
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_valid)

# Experimenting Starts HERE : 

0) :Naive-Bayes with tf-idf encoder 
1) : ANN
2) : LSTM 
3) : GRU
4) : Bidirectional LSTM
5) : 1D CNN
6) : Transfer Learning 1
7) : Transfer Learning 2
 

BASELINE model : Start with non deep learinng algorithm first

Create - > Fit - > Evaluate

In [23]:
from pyexpat import model
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
# from sklearn.naive_bayes import CategoricalNB

from sklearn.pipeline import Pipeline

# Pipeline is created for tokening and modelling
model_0 = Pipeline([
    ("tfidf" ,TfidfVectorizer()),# Convert words to numbers
    ("clf", BernoulliNB()) #model
])

model_0.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', BernoulliNB())])

In [24]:
baseline_score =  model_0.score(X_valid, y_valid)
print(f"Baseline Model score : {baseline_score}")

Baseline Model score : 0.991442542787286


In [57]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

def evaluate(y_true,y_pred):
    """ Calculate models accuracy, precesion, recall and f1score """
    # Basic Metrics
    print("Accuracy: in %", (accuracy_score(y_true, y_pred)*100))
    print("Precision:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall:", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score:", f1_score(y_true, y_pred, average='weighted'))

    # Detailed classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


In [34]:
y_pred = model_0.predict(X_valid)
y_true = y_valid
evaluate(y_true,y_pred)

Accuracy: in % 99.14425427872861
Precision: 0.9918847301182264
Recall: 0.991442542787286
F1 Score: 0.9914399966217787

Classification Report:
                          precision    recall  f1-score   support

            cancel_order       0.89      1.00      0.94        25
            change_order       0.97      0.92      0.95        39
 change_shipping_address       1.00      1.00      1.00        41
  check_cancellation_fee       1.00      1.00      1.00        26
           check_invoice       1.00      1.00      1.00        31
   check_payment_methods       1.00      1.00      1.00        26
     check_refund_policy       1.00      1.00      1.00        37
               complaint       0.96      1.00      0.98        23
contact_customer_service       1.00      1.00      1.00        24
     contact_human_agent       1.00      1.00      1.00        42
          create_account       1.00      0.92      0.96        25
          delete_account       1.00      1.00      1.00        29

# Lets try ANN model

In [24]:
from numpy import dtype
from tensorflow.keras import layers

#Functional API
input = layers.Input(shape=(1,), dtype = tf.string)# 1d strings
x = text_Vectorizer(input)
x = embedding(x)

# I added this to shrink (batch,tokens,embedding) to (batch,embedding) else due to its shape it will predict based on each token
x = layers.GlobalAveragePooling1D()(x)

outputs = layers.Dense(27, activation = "softmax")(x)
model_1 = tf.keras.Model(input,outputs,name = "ANN_model_1")

In [25]:
model_1.summary()

Model: "ANN_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 8)                0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 8, 128)            1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 27)                3483      
                                                                 
Total params: 1,283,483
Trainable params: 1,283,483
Non

In [26]:
model_1.compile(optimizer= tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics = ["accuracy"])

In [27]:
history_1 = model_1.fit(X_train,y_train_encoded,epochs = 10, validation_data=(X_valid,y_val_encoded))

Epoch 1/10
205/205 [==============================] - 10s 17ms/step - loss: 2.9411 - accuracy: 0.6917 - val_loss: 2.3255 - val_accuracy: 0.8240
Epoch 2/10
205/205 [==============================] - 3s 16ms/step - loss: 1.5600 - accuracy: 0.8616 - val_loss: 1.0051 - val_accuracy: 0.8826
Epoch 3/10
205/205 [==============================] - 2s 10ms/step - loss: 0.7224 - accuracy: 0.9090 - val_loss: 0.5805 - val_accuracy: 0.9010
Epoch 4/10
205/205 [==============================] - 2s 9ms/step - loss: 0.4615 - accuracy: 0.9232 - val_loss: 0.4344 - val_accuracy: 0.9095
Epoch 5/10
205/205 [==============================] - 2s 10ms/step - loss: 0.3545 - accuracy: 0.9277 - val_loss: 0.3643 - val_accuracy: 0.9144
Epoch 6/10
205/205 [==============================] - 2s 9ms/step - loss: 0.2985 - accuracy: 0.9309 - val_loss: 0.3266 - val_accuracy: 0.9193
Epoch 7/10
205/205 [==============================] - 2s 9ms/step - loss: 0.2651 - accuracy: 0.9329 - val_loss: 0.3016 - val_accuracy: 0.9156
E

In [28]:
len(y_train.unique())

27

In [29]:
model_1_preds = model_1.predict(X_valid)

26/26 [==============================] - 0s 4ms/step


In [30]:
model_1_preds.shape

(818, 27)

In [31]:
import numpy as np
model_1_pred_classes = np.argmax(model_1_preds, axis=1)

In [32]:
model_1_pred_classes

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 19,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  9,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  1, 25, 14,  2,
        2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  9, 25, 25,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  9,  4,  4,  4,  4,  4,  3,  4,  4,
        4,  4,  4,  3,  4,  4,  4,  4,  4,  5,  3,  9,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  5,  5,  5,  5,  5,  5,
        5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  3,  6,  6,  6,  9,  6,  6,
        6,  6,  6,  6,  7

In [35]:
model_1_results  = evaluate(y_true=y_val_encoded,y_pred=model_1_pred_classes)

Accuracy: in % 91.80929095354523
Precision: 0.9277048167739362
Recall: 0.9180929095354523
F1 Score: 0.9204209738853835

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        25
           1       0.95      1.00      0.97        39
           2       0.92      0.88      0.90        41
           3       0.79      0.88      0.84        26
           4       1.00      0.90      0.95        31
           5       0.96      0.85      0.90        26
           6       0.92      0.95      0.93        37
           7       0.96      0.96      0.96        23
           8       0.95      0.88      0.91        24
           9       0.83      0.93      0.88        42
          10       1.00      0.96      0.98        25
          11       1.00      0.90      0.95        29
          12       1.00      0.94      0.97        32
          13       0.97      0.97      0.97        38
          14       0.97      0.83      0.89   

In [39]:
model_1.save('my_model_1', save_format='tf')


INFO:tensorflow:Assets written to: my_model_1\assets


# Visualizing the embeddings

as we might know that the embedding layer initiates the vectors by setting it a random weights that are updated every iteration for better representation of the tokens

In [ ]:
embedding_layer = model_1.get_layer("embedding").get_weights()[0]
print(embedding_layer)
print(embedding_layer.shape)# Same shape as vocab size(every token) is embedded in 128 embedding_dim

[[-0.0666628   0.00545483  0.12573265 ...  0.00024062 -0.00515583
  -0.07189577]
 [ 0.04489614 -0.03666072  0.01595594 ... -0.00913266  0.00747273
  -0.00311805]
 [-0.09935233 -0.05252039  0.05597405 ... -0.11629315  0.0475892
   0.05027761]
 ...
 [-0.04647737 -0.00333396  0.00906645 ... -0.04611754  0.01078556
  -0.04616442]
 [-0.02692796  0.04989184  0.01555549 ...  0.006113    0.0312303
  -0.02017604]
 [ 0.03730098  0.00729885  0.02392875 ...  0.03894267 -0.00358186
   0.04672724]]
(10000, 128)


In [ ]:
import io
out_v = io.open('./projector_tf/vectors.tsv', 'w', encoding='utf-8') # Vectors ka tsv
out_m = io.open('./projector_tf/metadata.tsv', 'w', encoding='utf-8') # actual words ka TSV

for index, word in enumerate(words):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embedding_layer[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

# The concept is that similar words will have a closer embeddings which we can see using projector

Visualize using Projector.js

#  Model 3 RNN LSTM

input -> tokenize -> embedding -> layers (RNN/dense) -> Output

In [24]:
from numpy import shape
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype = tf.string)# 1d strings
x = text_Vectorizer(input)
x = embedding(x)
print(x.shape)
x = layers.LSTM(128,return_sequences=True)(x) # When stacking need to set return_sequences = True to prserve the dimensions (LSTM expects 3 dimensions)
print(x.shape)#(Batch, Timesteps, Features)
x = layers.LSTM(128,return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(32)(x)
print(x.shape)
x = layers.Dense(64,activation='relu')(x)
x = layers.Dense(32,activation='relu')(x)
print(x.shape)
outputs = layers.Dense(27, activation = "softmax")(x)

model_2 = tf.keras.Model(input, outputs, name="model_2_LSTM")

(None, 8, 128)
(None, 8, 128)
(None, 8, 128)
(None, 32)
(None, 32)


In [25]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 8)                0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 8, 128)            1280000   
                                                                 
 lstm (LSTM)                 (None, 8, 128)            131584    
                                                                 
 lstm_1 (LSTM)               (None, 8, 128)            131584    
                                                                 
 lstm_2 (LSTM)               (None, 32)                20608     
                                                      

In [26]:
model_2.compile(optimizer= tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics = ["accuracy"])

In [27]:
history_2 = model_2.fit(X_train,y_train_encoded,epochs = 10, validation_data=(X_valid,y_val_encoded))

Epoch 1/10
205/205 [==============================] - 10s 15ms/step - loss: 2.3694 - accuracy: 0.3069 - val_loss: 1.3282 - val_accuracy: 0.6540
Epoch 2/10
205/205 [==============================] - 2s 11ms/step - loss: 0.8274 - accuracy: 0.7949 - val_loss: 0.6153 - val_accuracy: 0.8435
Epoch 3/10
205/205 [==============================] - 2s 11ms/step - loss: 0.4555 - accuracy: 0.8821 - val_loss: 0.4861 - val_accuracy: 0.8802
Epoch 4/10
205/205 [==============================] - 2s 10ms/step - loss: 0.3393 - accuracy: 0.9090 - val_loss: 0.4194 - val_accuracy: 0.8936
Epoch 5/10
205/205 [==============================] - 2s 10ms/step - loss: 0.2883 - accuracy: 0.9182 - val_loss: 0.4052 - val_accuracy: 0.8900
Epoch 6/10
205/205 [==============================] - 2s 11ms/step - loss: 0.2836 - accuracy: 0.9177 - val_loss: 0.3923 - val_accuracy: 0.8961
Epoch 7/10
205/205 [==============================] - 2s 10ms/step - loss: 0.2552 - accuracy: 0.9241 - val_loss: 0.3592 - val_accuracy: 0.901

In [28]:
model_2_preds = model_2.predict(X_valid)
import numpy as np
model_2_pred_classes = np.argmax(model_2_preds, axis=1)

26/26 [==============================] - 1s 5ms/step


In [60]:
model_2_results = evaluate(y_true= y_val_encoded, y_pred= model_2_pred_classes)

Accuracy: in % 91.19804400977995
Precision: 0.9277166587943366
Recall: 0.9119804400977995
F1 Score: 0.9159118629227888

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        25
           1       1.00      1.00      1.00        39
           2       0.95      0.95      0.95        41
           3       0.70      0.88      0.78        26
           4       1.00      0.90      0.95        31
           5       1.00      0.85      0.92        26
           6       0.92      0.95      0.93        37
           7       0.96      0.96      0.96        23
           8       1.00      0.92      0.96        24
           9       0.95      0.90      0.93        42
          10       0.96      0.96      0.96        25
          11       0.96      0.90      0.93        29
          12       1.00      0.94      0.97        32
          13       0.97      0.97      0.97        38
          14       0.97      0.83      0.89   

In [29]:
model_2.save('./model/my_model_2', save_format='tf')

INFO:tensorflow:Assets written to: ./model/my_model_2\assets


INFO:tensorflow:Assets written to: ./model/my_model_2\assets


# Now lets try GRU 

It has relatively less parameters

In [48]:
from tensorflow.keras import layers
import tensorflow as tf

input = layers.Input(shape=(1,),dtype=tf.string)
x = text_Vectorizer(input)
x = embedding(x)
x = layers.GRU(128,return_sequences= True)(x) # Simplyfing model to reduce overfitting
# x = layers.GRU(128, return_sequences= True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(27, activation = "softmax")(x)

model_3 = tf.keras.Model(input, outputs, name="model_3_GRU")

In [49]:
model_3.compile(optimizer= tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics = ["accuracy"])

In [50]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 8)                0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 8, 128)            1280000   
                                                                 
 gru_13 (GRU)                (None, 8, 128)            99072     
                                                                 
 gru_14 (GRU)                (None, 64)                37248     
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                       

In [51]:
history_3 = model_3.fit(X_train,y_train_encoded,epochs = 10, validation_data=(X_valid,y_val_encoded))

Epoch 1/10
205/205 [==============================] - 5s 11ms/step - loss: 0.9613 - accuracy: 0.7804 - val_loss: 0.3632 - val_accuracy: 0.8998
Epoch 2/10
205/205 [==============================] - 2s 8ms/step - loss: 0.2383 - accuracy: 0.9353 - val_loss: 0.2787 - val_accuracy: 0.9120
Epoch 3/10
205/205 [==============================] - 2s 8ms/step - loss: 0.2068 - accuracy: 0.9350 - val_loss: 0.2648 - val_accuracy: 0.9205
Epoch 4/10
205/205 [==============================] - 2s 8ms/step - loss: 0.1945 - accuracy: 0.9378 - val_loss: 0.2741 - val_accuracy: 0.9193
Epoch 5/10
205/205 [==============================] - 2s 8ms/step - loss: 0.1927 - accuracy: 0.9368 - val_loss: 0.2580 - val_accuracy: 0.9144
Epoch 6/10
205/205 [==============================] - 2s 8ms/step - loss: 0.1834 - accuracy: 0.9356 - val_loss: 0.2653 - val_accuracy: 0.9181
Epoch 7/10
205/205 [==============================] - 2s 8ms/step - loss: 0.1806 - accuracy: 0.9367 - val_loss: 0.2597 - val_accuracy: 0.9218
Epoch

In [52]:
model_3.save('./model/my_model_3', save_format='tf')

INFO:tensorflow:Assets written to: ./model/my_model_3\assets


INFO:tensorflow:Assets written to: ./model/my_model_3\assets


In [54]:
model_3_pred = model_3.predict(X_valid)

26/26 [==============================] - 0s 5ms/step


In [58]:
import numpy as np
model_3_pred_classes = np.argmax(model_3_pred, axis=1)
model_3_results = evaluate(y_true= y_val_encoded, y_pred= model_3_pred_classes)

Accuracy: in % 90.95354523227384
Precision: 0.926548449437323
Recall: 0.9095354523227384
F1 Score: 0.9132363574104877

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        25
           1       0.93      0.97      0.95        39
           2       0.89      0.95      0.92        41
           3       0.83      0.77      0.80        26
           4       1.00      0.90      0.95        31
           5       1.00      0.85      0.92        26
           6       0.92      0.95      0.93        37
           7       0.96      0.96      0.96        23
           8       1.00      0.88      0.93        24
           9       0.82      0.95      0.88        42
          10       1.00      0.92      0.96        25
          11       0.96      0.90      0.93        29
          12       0.94      0.97      0.95        32
          13       1.00      0.89      0.94        38
          14       1.00      0.86      0.92    

LSTM and GRU performs relatively simmilar according to research papers 

# Lets try a Bidirectional RNN

In [77]:
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_Vectorizer(input)
x = embedding(x)
x = layers.Bidirectional(layer= layers.GRU(128, return_sequences= True))(x)
x = layers.Bidirectional(layer= layers.LSTM(64))(x)
x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(27, activation = "softmax")(x)

model_4 = tf.keras.Model(input, outputs, name="model_4_BIRNN")

In [78]:
model_4.compile(optimizer= 'Adam',loss= tf.keras.losses.sparse_categorical_crossentropy, metrics= ['Accuracy'])

In [79]:
model_4.summary()

Model: "model_4_BIRNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 8)                0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 8, 128)            1280000   
                                                                 
 bidirectional_9 (Bidirectio  (None, 8, 256)           198144    
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 128)              164352    
 onal)                                                           
                                                     

In [80]:
history_4 = model_4.fit(X_train,y_train_encoded,epochs = 10, validation_data=(X_valid,y_val_encoded))

Epoch 1/10
205/205 [==============================] - 7s 17ms/step - loss: 0.7479 - Accuracy: 0.8280 - val_loss: 0.2918 - val_Accuracy: 0.9059
Epoch 2/10
205/205 [==============================] - 2s 11ms/step - loss: 0.2086 - Accuracy: 0.9345 - val_loss: 0.2874 - val_Accuracy: 0.9120
Epoch 3/10
205/205 [==============================] - 2s 11ms/step - loss: 0.2028 - Accuracy: 0.9330 - val_loss: 0.2770 - val_Accuracy: 0.9193
Epoch 4/10
205/205 [==============================] - 2s 12ms/step - loss: 0.1864 - Accuracy: 0.9375 - val_loss: 0.2675 - val_Accuracy: 0.9144
Epoch 5/10
205/205 [==============================] - 2s 11ms/step - loss: 0.1846 - Accuracy: 0.9381 - val_loss: 0.2586 - val_Accuracy: 0.9169
Epoch 6/10
205/205 [==============================] - 2s 11ms/step - loss: 0.1797 - Accuracy: 0.9384 - val_loss: 0.2594 - val_Accuracy: 0.9181
Epoch 7/10
205/205 [==============================] - 2s 11ms/step - loss: 0.1749 - Accuracy: 0.9378 - val_loss: 0.2761 - val_Accuracy: 0.9156

In [81]:
model_4.save('./model/my_model_4', save_format='tf')

INFO:tensorflow:Assets written to: ./model/my_model_4\assets


INFO:tensorflow:Assets written to: ./model/my_model_4\assets


In [85]:
model_4_pred = model_4.predict(X_valid)
model_4_pred_class = np.argmax(model_4_pred, axis= 1)
model_3_results = evaluate(y_true= y_val_encoded, y_pred= model_3_pred_classes)

26/26 [==============================] - 0s 7ms/step
Accuracy: in % 90.95354523227384
Precision: 0.926548449437323
Recall: 0.9095354523227384
F1 Score: 0.9132363574104877

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        25
           1       0.93      0.97      0.95        39
           2       0.89      0.95      0.92        41
           3       0.83      0.77      0.80        26
           4       1.00      0.90      0.95        31
           5       1.00      0.85      0.92        26
           6       0.92      0.95      0.93        37
           7       0.96      0.96      0.96        23
           8       1.00      0.88      0.93        24
           9       0.82      0.95      0.88        42
          10       1.00      0.92      0.96        25
          11       0.96      0.90      0.93        29
          12       0.94      0.97      0.95        32
          13       1.00      0.89      0.94     